In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.2. Biomarker Distribution Analysis/Output')
cur_date = "050423"

library(readxl)
library(tidyverse)
library(reshape2)

# reading in files
cytokine_df = data.frame(read_excel("Input/Cytokine_Data_050423.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message:
“Expecting numeric in E1226 / R1226C5: got 'NA'”
Warning message:
“Expecting numeric in E1227 / R1227C5: got 'NA'”
Warning message:
“Expecting numeric in E1228 / R1228C5: got 'NA'”
Warning message:
“Expecting numeric in E1229 / R1229C5: got 'NA'”
Warning message:
“Expecting numeric in E1230 / R1230C5: got 'NA'”
Warning message:
“Expecting numeric in E1231 / R1231C5: got 'NA'”
Warning message:
“Expecting numeric in E1232 / R1232C5: got 'NA'”
Warning message:
“Expecting numeric in E1233 / 

In [2]:
head(cytokine_df)

,Subject_No,Subject_ID,Condensate,Burn_Condition,Condensate_Conc,Cytokine,Cytokine_Conc,Cytokine_Conc_pslog2
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
1,1,F_1,C,F,1,Eotaxin3,21.750958,4.507855
2,1,F_1,C,F,25,Eotaxin3,11.624224,3.658123
3,1,F_1,C,F,5,Eotaxin3,17.828714,4.234863
4,1,F_1,C,S,1,Eotaxin3,22.798534,4.572801
5,1,F_1,C,S,25,Eotaxin3,10.342567,3.503675
6,1,F_1,C,S,5,Eotaxin3,9.140652,3.342078


Testing for statistical differences by comparing a cytokine concentration between burn conditions (control, smoldering or flaming). (The condensate and concentration would remain consistent). Cytokine concentration was pseudo log transformed to reduce skew.

Typically, normality and homogeneity of variances would be tested for first, however a non-parametric test (Wilcoxon rank sum) will be used given the small sample size (n = 6).

In [3]:
wilcoxon_rank_sumv1 = function(df){
    # """
    # Running wilcoxon rank sums after filtering for a cytokine and condensate using a loop. 
    # Ultimately using this test to compare cytokine expression (control vs. flaming burn condition).

    # :param: cytokine dataframe
    # :output: a dataframe containing the cytokine, condensate, comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    cytokines = unique(df$Cytokine)
    condensates = c("C", "P")
    conc = unique(df$Condensate_Conc)[1:3]
    
    values_df = data.frame()
     # iterating through each cytokine, condensate, and concentration
    for(i in 1:length(cytokines)){
        for(j in 1:length(condensates)){
            for(k in 1:length(conc)){
                
                # control df
                control_df = df %>%
                    filter(Cytokine == cytokines[i], Condensate == "PBS")

                # flaming df
                flaming_df = df %>%
                    filter(Cytokine == cytokines[i], Condensate == condensates[j], Burn_Condition == "F", 
                           Condensate_Conc == conc[k])

                # wilcoxon rank sum
                wilcoxon_test = wilcox.test(control_df$Cytokine_Conc_pslog2, flaming_df$Cytokine_Conc_pslog2,
                                            paired = TRUE)

                # contains cytokine, condensate, burn conditions, conc, stat, and p value
                values_vector = cbind(cytokines[i], condensates[j], "Control vs. Flaming", conc[k], 
                                      wilcoxon_test$statistic, wilcoxon_test$p.value)
                values_df = rbind(values_df, values_vector)
            }
        }
    }

    
    # adding col names
    colnames(values_df) = c("Cytokine", "Condensate", "Comparison", "Concentration", "Statistic", "P Value")
    
    # calculating padj values
    values_df$`P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")

    
    return(values_df)
}

# calling fn
control_wilcox_values = wilcoxon_rank_sumv1(cytokine_df)

In [4]:
wilcoxon_rank_sumv2 = function(df){
    # """
    # Running wilcoxon rank sums after filtering for a cytokine and condensate using a loop. 
    # Ultimately using this test to compare cytokine expression (control vs. flaming burn condition).

    # :param: cytokine dataframe
    # :output: a dataframe containing the cytokine, condensate, comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    cytokines = unique(df$Cytokine)
    condensates = c("C", "P")
    conc = unique(df$Condensate_Conc)[1:3]
    
    values_df = data.frame()
     # iterating through each cytokine, condensate, and concentration
    for(i in 1:length(cytokines)){
        for(j in 1:length(condensates)){
            for(k in 1:length(conc)){
                
                # control df
                control_df = df %>%
                    filter(Cytokine == cytokines[i], Condensate == condensates[j], Burn_Condition == "S",
                          Condensate_Conc == conc[k])

                # flaming df
                flaming_df = df %>%
                    filter(Cytokine == cytokines[i], Condensate == condensates[j], Burn_Condition == "F", 
                           Condensate_Conc == conc[k])

                # wilcoxon rank sum
                wilcoxon_test = wilcox.test(control_df$Cytokine_Conc_pslog2, flaming_df$Cytokine_Conc_pslog2,
                                            paired = TRUE)

                # contains cytokine, condensate, burn conditions, conc, stat, and p value
                values_vector = cbind(cytokines[i], condensates[j], "Smoldering vs. Flaming", conc[k], 
                                      wilcoxon_test$statistic, wilcoxon_test$p.value)
                values_df = rbind(values_df, values_vector)
            }
        }
    }

    
    # adding col names
    colnames(values_df) = c("Cytokine", "Condensate", "Comparison", "Concentration", "Statistic", "P Value")
    
    # calculating padj values
    values_df$`P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")

    
    return(values_df)
}

# calling fn
smoldering_wilcox_values = wilcoxon_rank_sumv2(cytokine_df)

In [5]:
# creating 1 df
cytokine_wilcoxon_df = rbind(control_wilcox_values, smoldering_wilcox_values) %>%
    # changing condensates so they're more legible
    mutate(Condensate = ifelse(Condensate == "C", "Cardboard",
                               ifelse(Condensate == "P", "Plastic", "PBS")))

head(cytokine_wilcoxon_df)

,Cytokine,Condensate,Comparison,Concentration,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
V,Eotaxin3,Cardboard,Control vs. Flaming,1,6,0.4375,0.7828947
V1,Eotaxin3,Cardboard,Control vs. Flaming,25,12,0.84375,0.9354620
V2,Eotaxin3,Cardboard,Control vs. Flaming,5,9,0.84375,0.9354620
V3,Eotaxin3,Plastic,Control vs. Flaming,1,11,1,1.0000000
V4,Eotaxin3,Plastic,Control vs. Flaming,25,9,0.84375,0.9354620
V5,Eotaxin3,Plastic,Control vs. Flaming,5,16,0.3125,0.7589286


In [6]:
# exporting
write.csv(cytokine_wilcoxon_df, paste0(Output,"/", cur_date, "_Cytokine_Wilcoxon_Results.csv"), row.names = FALSE)